In [ ]:
from combineDatasets import extract_features_elevation , extract_features_landcover , extract_features_monthly_clim , extract_features_soil , organize_monthly_climat_files
from scripts.tools.merge_data_sources import merge_data_sources





# Merging Data

## Climat Dataset



In [ ]:

# Organize the files
monthly_tmax_data = organize_monthly_climat_files(
    "../data/climate_dataset/5min/max/*.tif"
)
monthly_tmin_data = organize_monthly_climat_files(
    "../data/climate_dataset/5min/min/*.tif"
)
monthly_tprec_data = organize_monthly_climat_files(
    "../data/climate_dataset/5min/prec/*.tif"
)


fires_tmax = extract_features_monthly_clim(
    fire_csv="../data/fire_dataset/viirs-jpss1_2024_alg_Tun.csv",
    raster_dict=monthly_tmax_data,
    output_path="../data/features/fire_tmax.csv",
    value_name="tmax",
)


fires_tmin = extract_features_monthly_clim(
    fire_csv="../data/fire_dataset/viirs-jpss1_2024_alg_Tun.csv",
    raster_dict=monthly_tmin_data,
    output_path="../data/features/fire_tmin.csv",
    value_name="tmin",
)


fires_tprec = extract_features_monthly_clim(
    fire_csv="../data/fire_dataset/viirs-jpss1_2024_alg_Tun.csv",
    raster_dict=monthly_tprec_data,
    output_path="../data/features/fire_tprec.csv",
    value_name="tprec",
)

TypeError: raster_to_dataframe_filter_background() missing 1 required positional argument: 'src_crs'

## Landcover

In [ ]:

fires_with_landcover = extract_features_landcover(
    csv_path="../data/fire_dataset/viirs-jpss1_2024_alg_Tun.csv",
    shapefile_path="../data/land_dataset/combined/alg_tun_landcvr.shx",
    lat_col="latitude",
    lon_col="longitude",
    keep_cols=["GRIDCODE"],  # can be ["GRIDCODE", "CLASS", "AREA", ...]
    output_path="../data/features/fire_landcover.csv",
)


## SOil

In [ ]:
fires_with_soil = extract_features_soil(
    csv_path="../data/fire_dataset/viirs-jpss1_2024_alg_Tun.csv",
    raster_path="../data/soil_dataset/HWSD2_RASTER/HWSD2.bil",
    output_soil_ids="../data/features/fire_soil_ids.csv",
    output_soil_feature="../data/features/fire_soil.csv",
)

## elevation

In [ ]:
fires_with_elevation = extract_features_elevation(
    raster_path="../data/elevation_dataset/elevation_clipped.tif",
    fire_csv_path="../data/fire_dataset/viirs-jpss1_2024_alg_Tun.csv",
    output_csv="../data/features/fire_elevation.csv",
    value_name="elevation",
)


## Merging

In [ ]:
csv_list= ["../data/fire_dataset/viirs-jpss1_2024_alg_Tun.csv", "../data/features/fire_landcover.csv" , "../data/features/fire_elevation.csv" , "../data/features/fire_soil.csv"]
temp_df = merge_data_sources(csv_list , ["latitude","longitude"], "inner")
sources = [temp_df , "../data/features/fire_tmin.csv" , "../data/features/fire_tmax.csv" , "../data/features/fire_tprec.csv"]
merge_data_sources(csv_list , ["latitude","longitude", "acq_date"], "inner", "../data/Merged/")